In [1]:
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.models.google_llm import Gemini

In [2]:
retry_config=types.HttpRetryOptions(attempts=3, exp_base=5, initial_delay=1, http_status_codes=[429, 500, 503, 504])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

In [ ]:
def get_grades(student_name: str) -> str:
    """Get grades of a spefcified student, from Math, History and Science.

    Args:
        student_name (str): first name of the student

    Returns:
        Dictionary with status and a string with one grade per each subject
        Success: {"status": "success", "grades": "Math: X, History: Y, Science: Z"}
        Error: {"status": "error", "error_message": "..."}
    """
    grades = {
        "Alice": "Math: A, History: B, Science: A",
        "Bob": "Math: C, History: A, Science: B",
        "Charlie": "Math: B, History: C, Science: A"
    }
    student_grades = grades.get(student_name)
    if student_grades:
        return {"status": "success", "grades": student_grades}
    return {"status": "error", "message": "Student not found"}

In [4]:
teacher_agent = Agent(
    name="TeacherAgent",
    model=model,
    instruction="""
    You are a teacher on a parent-teacher conference.
    Answer the questions about my children's progress in school.
    Use get_grades tool to fetch student's grades.
    """,
    tools=[get_grades],
)

In [7]:
runner = InMemoryRunner(agent=teacher_agent)
_ = await runner.run_debug("What is Bob's grades?")


 ### Created new session: debug_session_id

User > What is Bob's grades?
TeacherAgent > The student's grades are: Math: C, History: A, Science: B.
